In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib
import math

# Data Collection and Dataframe Creation

In [2]:
pd.options.display.max_columns = None

In [23]:
# Get the league ID from the user
league_id = input("Enter the league ID: ")

# Create the URL using an f-string
url = f'https://fantasy.premierleague.com/api/leagues-classic/{league_id}/standings/'

try:
    # Make the request and parse the response as JSON
    response = requests.get(url)
    response.raise_for_status()  # Check for HTTP request errors
    data = response.json()
except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")
except requests.exceptions.JSONDecodeError:
    print("The response is not a valid JSON document.")

In [24]:
#Extract data for the DataFrame
new_entries = data.get('new_entries', {})
last_updated_data = data.get('last_updated_data', None)
league = data.get('league', {})
standings = data.get('standings', {}).get('results', [])

# Create DataFrames for each key if they are not empty
if new_entries:
    df_new_entries = pd.DataFrame(new_entries.get('results', []))
else:
    df_new_entries = pd.DataFrame()

if league:
    df_league = pd.DataFrame([league])  # Convert dictionary to a DataFrame with one row
else:
    df_league = pd.DataFrame()

if standings:
    df_standings = pd.DataFrame(standings)
else:
    df_standings = pd.DataFrame()

# Extract the columns of interest
columns_of_interest = ['entry', 'entry_name', 'rank', 'player_name', 'total']
df_filtered_standings = df_standings[columns_of_interest]

# Display the filtered standings DataFrame
print("Filtered Standings DataFrame:")
print(df_filtered_standings)

Filtered Standings DataFrame:
      entry            entry_name  rank      player_name  total
0    296696       The Special One     1      Theo Murray   2421
1   2563745        Rice Rice Baby     2      Jack Murray   2351
2   3190130               Macleod     3     Andy Macleod   2254
3   2118759       The Hucklebucks     4     Simon Daniel   2236
4    161957      poor last season     5     Ross Laverty   2215
5   2078027       Rashford Fiesta     6  Ritchie Laverty   2197
6   2082148   7th place last time     7     Louis Murray   2123
7   1562151    MK Munchengladbach     8       Ken Murray   2103
8   2077579    Grealish Lightning     9      Iain Murray   2094
9   2683089     Super John McGinn    10  Stephen Macleod   2093
10  2330200           Change Name    11      Euan Murray   2064
11  1636610  We've got Egan-Riley    12   Daniel Macleod   2055
12  2710147            Tubeventus    13     John Macleod   2003
13  8011260         Cearns Cosmos    14    Gordon Murray   1771
14  564268

In [25]:
# Function to get the history for a given team id
def fetch_team_history(team_id):
    url = f'https://fantasy.premierleague.com/api/entry/{team_id}/history/'
    response = requests.get(url)
    response.raise_for_status()  # Check for HTTP request errors
    return response.json()

# Initialize a list to store the results
all_histories = []

# Loop through each entry in the filtered standings DataFrame
for team_id in df_filtered_standings['entry']:
    try:
        history_data = fetch_team_history(team_id)
        # Flatten the JSON and add the team id for reference
        for history in history_data.get('current', []):
            history['entry'] = team_id
            all_histories.append(history)
    except requests.exceptions.RequestException as e:
        print(f"Request error for team id {team_id}: {e}")

# Create a new DataFrame from the collected history data
df_team_histories = pd.DataFrame(all_histories)

# Extract only the columns of interest
columns_of_interest = ['entry', 'event', 'points', 'total_points', 'value', 'points_on_bench']
df_team_histories_filtered = df_team_histories[columns_of_interest]

# Set 'entry' as the primary key
df_team_histories_filtered.set_index('entry', inplace=True)

# Display the new DataFrame
print("Filtered Team Histories DataFrame:")
print(df_team_histories_filtered)

Filtered Team Histories DataFrame:
         event  points  total_points  value  points_on_bench
entry                                                       
296696       1      73            73   1000                6
296696       2      40           113   1001               11
296696       3      45           158   1006                8
296696       4      87           245   1007                4
296696       5      64           309   1008                9
...        ...     ...           ...    ...              ...
5642687     34      62          1514    988                0
5642687     35      51          1565    987               10
5642687     36      78          1643    987                3
5642687     37      81          1724    986                8
5642687     38      39          1763    988                1

[570 rows x 5 columns]


In [26]:
# Extracting league name from df_league DataFrame
league_name = df_league.loc[0, 'name']

# Assign league name to variable 'League Name'
League_Name = league_name

# Displaying league name
print("League Name:", League_Name)

League Name: The worst league ever part 2


# Graph Creation

# PDF Creation

In [27]:
pip install reportlab

Note: you may need to restart the kernel to use updated packages.


In [28]:
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, PageBreak, Table, TableStyle
from reportlab.lib import colors
import pandas as pd

# Function to generate PDF
def generate_pdf(league_name, df_filtered_standings):
    # Set up PDF document
    filename = f"{league_name}.pdf"
    doc = SimpleDocTemplate(filename, pagesize=letter)

    # Initialize story and individual pages list
    story = []
    individual_pages = []

    # Title Page
    title_style = ParagraphStyle(name='Title', fontSize=18, alignment=1, spaceAfter=30, fontName='Helvetica-Bold')
    subtitle_style = ParagraphStyle(name='Subtitle', fontSize=14, alignment=1, spaceAfter=30)

    title_text = "Fantasy Premier League Analysis - Classic League"
    subtitle_text = league_name

    title_paragraph = Paragraph(title_text, title_style)
    subtitle_paragraph = Paragraph(subtitle_text, subtitle_style)

    story.append(title_paragraph)
    story.append(subtitle_paragraph)
    story.append(PageBreak())

    # Prepare data for the table
    headers = ['Player Name', 'Entry Name', 'Total']
    data = []

    for index, row in df_filtered_standings.iterrows():
        player_name = row['player_name']
        entry_name = row['entry_name']
        total = row['total']

        # Add data row to table
        data.append([player_name, entry_name, total])

        # Create individual page for each player
        player_key = f"{player_name} - {entry_name}"
        player_page = [
            Paragraph(f"<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>{player_key}", title_style),
            PageBreak()
        ]
        individual_pages.append(player_page)

    # Create table
    table_data = []
    for row in data:
        player_name, entry_name, total = row
        table_data.append([player_name, entry_name, total])

    table = Table([headers] + table_data)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))

    story.append(table)
    story.append(PageBreak())

    # Add each individual player's page
    for page in individual_pages:
        story.extend(page)

    # Build PDF document
    doc.build(story)

# Example usage:
if __name__ == "__main__":
    # Assuming df_filtered_standings and league_name are already defined
    generate_pdf(league_name, df_filtered_standings)